In [1]:
import pandas as pd
import numpy as np
import datetime
import calendar
import scipy.signal
import ipywidgets

In [2]:
START_CASES = 500
COUNTRY = 'Finland'

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
data = data[data['Country/Region'] == COUNTRY].loc[:, '1/22/20':].T.iloc[:, 0].rename('cum_cases')
data.index = pd.to_datetime(data.index).rename('date')
# start from first day with at most START_CASES cases
data = data[(data >= START_CASES).idxmax():]

In [4]:
data

date
2020-03-21      523
2020-03-22      626
2020-03-23      700
2020-03-24      792
2020-03-25      880
              ...  
2021-03-18    69497
2021-03-19    70267
2021-03-20    71123
2021-03-21    71643
2021-03-22    72073
Name: cum_cases, Length: 367, dtype: int64

In [5]:
%matplotlib widget
data.plot(title='Cases, cumulative');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
cases_per_day = data.diff().dropna()

%matplotlib widget
cases_per_day.plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
%matplotlib widget

for weekday in range(7):
    # df = nonzero_days[nonzero_days.index.weekday == weekday]
    df = cases_per_day[cases_per_day.index.weekday == weekday]
    df.rolling(5, center=True).mean().plot(label=calendar.day_name[weekday], legend=True, logy=True, title='Daily new cases, weekdays, smoothed')
    # df.plot(label=calendar.day_name[weekday], legend=True, title='Daily new cases')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget

MIN_CASES_PER_DAY = 0
smoothed = cases_per_day.rolling(7, center=True).mean()
smoothed = smoothed.where(smoothed >= MIN_CASES_PER_DAY)
diff_to_win = (cases_per_day - smoothed) / smoothed * 100.0
for weekday in range(7):
    df = diff_to_win[cases_per_day.index.weekday == weekday]
    df.rolling(5, center=True).mean().plot(label=calendar.day_name[weekday], legend=True, logy=False, title='Daily new cases, diff to 7-day window mean, %')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
%matplotlib widget
np.log(cases_per_day.where(cases_per_day > 0)).diff().plot(title='Change in number of new cases from previous day');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget
# Very small cases_per_day poses problems because we use log; interpolate those values
log_new_cases = np.log(cases_per_day.where(cases_per_day > 2)).interpolate().dropna()
log_new_cases.plot(title='log(daily new cases)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
f, P_density = scipy.signal.periodogram(cases_per_day.diff().dropna(), window='hann', scaling='spectrum')
pg = pd.Series(data=P_density[f > 0], index=1.0/f[f > 0])

In [12]:
%matplotlib widget
pg.plot(xlim=(1.5, 15), title='Spectrum of cases-per-day-change', xlabel='period (days)', ylabel='amplitude');
# pg.plot(xlim=(1.5, 15), title='Spectrum of cases-per-day-change', xlabel='period (days)', ylabel='amplitude', logy=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def design_filter(cutoff_period, order=16):
    filt_sos = scipy.signal.butter(order, 1/cutoff_period, output='sos', fs=1.0)
    return filt_sos

In [14]:
%matplotlib widget
cases_per_day_filt = pd.Series(scipy.signal.sosfiltfilt(design_filter(10.0), cases_per_day), index=cases_per_day.index)
cases_per_day.plot(title='Daily new cases', label='Unfiltered', legend=True, figsize=(9, 4), alpha=.5)
ax = cases_per_day_filt.plot(label='Filtered', legend=True, color='red')

line = ax.lines[-1]

def plot_filtered(period=10, filter_order=16, peek_into_future=True):
    sos = design_filter(period, order=filter_order)
    filt_fun = scipy.signal.sosfilt
    if peek_into_future:
        filt_fun = scipy.signal.sosfiltfilt
    cases_per_day_filt = pd.Series(filt_fun(sos, cases_per_day), index=cases_per_day.index)
    line.set_ydata(cases_per_day_filt)
    ax.figure.canvas.draw_idle()
    
layout = {'width': 'auto'}
    
ipywidgets.interactive(
    plot_filtered,
    period=ipywidgets.IntSlider(min=3, max=90, value=10, layout=layout),
    filter_order=ipywidgets.IntSlider(min=1, max=32, value=8, layout=layout),
    peek_into_future=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=10, description='period', layout=Layout(width='auto'), max=90, min=3), I…